<a href="https://colab.research.google.com/github/AnkurMali/IST597_SP21/blob/main/SP21_IST597_week5_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IST597:- Multi-Layer Perceptron

## Load the libraries

In [4]:
import os
import numpy as np
import time
import tensorflow as tf


In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

#Get number of Gpu's and id's in the system or else you can also use Nvidia-smi in command prompt.

## Generate random data

In [6]:
size_input = 32
size_hidden = 128
size_output = 1
number_of_train_examples = 1000
number_of_test_examples = 300

In [7]:
X_train = np.random.randn(number_of_train_examples , size_input)
y_train = np.random.randn(number_of_train_examples)
X_test = np.random.randn(number_of_test_examples, size_input)
y_test = np.random.randn(number_of_test_examples)

In [8]:
# Split dataset into batches
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(16)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(4)

## Build MLP using Eager Execution

In [9]:
# Define class to build mlp model
class MLP(object):
  def __init__(self, size_input, size_hidden, size_output, device=None):
    """
    size_input: int, size of input layer
    size_hidden: int, size of hidden layer
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
    """
    self.size_input, self.size_hidden, self.size_output, self.device =\
    size_input, size_hidden, size_output, device
    
    # Initialize weights between input layer and hidden layer
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden]))
    # Initialize biases for hidden layer
    self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden]))
     # Initialize weights between hidden layer and output layer
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden, self.size_output]))
    # Initialize biases for output layer
    self.b2 = tf.Variable(tf.random.normal([1, self.size_output]))
    
    # Define variables to be updated during backpropagation
    self.variables = [self.W1, self.W2, self.b1, self.b2]
    
  def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """
    if self.device is not None:
      with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
      
    return self.y
  
  def loss(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape (batch_size, size_output)
    y_true - Tensor of shape (batch_size, size_output)
    '''
    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    return tf.losses.mean_squared_error(y_true_tf, y_pred_tf)
  
  def backward(self, X_train, y_train):
    """
    backward pass
    """
    optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4)
    with tf.GradientTape() as tape:
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients(zip(grads, self.variables))
        
        
  def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """
    # Cast X to float32
    X_tf = tf.cast(X, dtype=tf.float32)
    #Remember to normalize your dataset before moving forward
    # Compute values in hidden layer
    what = tf.matmul(X_tf, self.W1) + self.b1
    hhat = tf.nn.relu(what)
    # Compute output
    output = tf.matmul(hhat, self.W2) + self.b2
    #Now consider two things , First look at inbuild loss functions if they work with softmax or not and then change this
    #Second add tf.Softmax(output) and then return this variable
    return output

## Train Model

In [10]:
# Set number of epochs
NUM_EPOCHS = 10

In [24]:
# Initialize model using CPU
mlp_on_cpu = MLP(size_input, size_hidden, size_output, device='cpu')

time_start = time.time()
for epoch in range(NUM_EPOCHS):
  loss_total = tf.zeros([1,1], dtype=tf.float32)
  lt = 0
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(25, seed=epoch*(1234)).batch(20)
  for inputs, outputs in train_ds:
    preds = mlp_on_cpu.forward(inputs)
    loss_total = loss_total + mlp_on_cpu.loss(preds, outputs)
    lt = lt + mlp_on_cpu.loss(preds, outputs)
    mlp_on_cpu.backward(inputs, outputs)
  print('Number of Epoch = {} - Average MSE:= {}'.format(epoch + 1, np.sum(loss_total) / X_train.shape[0]))
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))
#For per epoch_time = Total_Time / Number_of_epochs

Number of Epoch = 1 - Average MSE:= 9105.663
Number of Epoch = 2 - Average MSE:= 37.10768359375
Number of Epoch = 3 - Average MSE:= 17.267142578125
Number of Epoch = 4 - Average MSE:= 10.67531640625
Number of Epoch = 5 - Average MSE:= 7.65931689453125
Number of Epoch = 6 - Average MSE:= 5.80377978515625
Number of Epoch = 7 - Average MSE:= 4.49806591796875
Number of Epoch = 8 - Average MSE:= 3.750736328125
Number of Epoch = 9 - Average MSE:= 3.199549560546875
Number of Epoch = 10 - Average MSE:= 2.821003662109375

Total time taken (in seconds): 4.37


In [27]:
# Initialize model using GPU
mlp_on_gpu = MLP(size_input, size_hidden, size_output, device='gpu')

time_start = time.time()
for epoch in range(NUM_EPOCHS):
  loss_total_gpu = tf.zeros([1,1], dtype=tf.float32)
  lt = 0
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(25, seed=epoch*(1234)).batch(20)
  for inputs, outputs in train_ds:
    preds = mlp_on_gpu.forward(inputs)
    loss_total_gpu = loss_total_gpu + mlp_on_gpu.loss(preds, outputs)
    lt = lt + mlp_on_gpu.loss(preds, outputs)
    mlp_on_gpu.backward(inputs, outputs)
  print('Number of Epoch = {} - Average MSE:= {}'.format(epoch + 1, np.sum(loss_total_gpu) / X_train.shape[0]))
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

Number of Epoch = 1 - Average MSE:= 5028.0235
Number of Epoch = 2 - Average MSE:= 30.274162109375
Number of Epoch = 3 - Average MSE:= 13.53579296875
Number of Epoch = 4 - Average MSE:= 8.2266064453125
Number of Epoch = 5 - Average MSE:= 5.69138134765625
Number of Epoch = 6 - Average MSE:= 4.03516748046875
Number of Epoch = 7 - Average MSE:= 3.119510498046875
Number of Epoch = 8 - Average MSE:= 2.475406005859375
Number of Epoch = 9 - Average MSE:= 2.14405126953125
Number of Epoch = 10 - Average MSE:= 1.8120531005859375

Total time taken (in seconds): 3.09


In [28]:
#Default mode
mlp_on_default = MLP(size_input, size_hidden, size_output)

time_start = time.time()
for epoch in range(NUM_EPOCHS):
  loss_total_gpu = tf.zeros([1,1], dtype=tf.float32)
  lt = 0
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(25, seed=epoch*(1234)).batch(20)
  for inputs, outputs in train_ds:
    preds = mlp_on_default.forward(inputs)
    loss_total_gpu = loss_total_gpu + mlp_on_default.loss(preds, outputs)
    lt = lt + mlp_on_default.loss(preds, outputs)
    mlp_on_default.backward(inputs, outputs)
  print('Number of Epoch = {} - Average MSE:= {}'.format(epoch + 1, np.sum(loss_total_gpu) / X_train.shape[0]))
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

Number of Epoch = 1 - Average MSE:= 5260.6275
Number of Epoch = 2 - Average MSE:= 29.074205078125
Number of Epoch = 3 - Average MSE:= 15.23725390625
Number of Epoch = 4 - Average MSE:= 9.720046875
Number of Epoch = 5 - Average MSE:= 6.98215283203125
Number of Epoch = 6 - Average MSE:= 5.27155859375
Number of Epoch = 7 - Average MSE:= 4.107587890625
Number of Epoch = 8 - Average MSE:= 3.36253857421875
Number of Epoch = 9 - Average MSE:= 2.834361572265625
Number of Epoch = 10 - Average MSE:= 2.49869873046875

Total time taken (in seconds): 3.11


In [29]:
#TPU mode
mlp_on_gpu = MLP(size_input, size_hidden, size_output, device='tpu')

time_start = time.time()
for epoch in range(NUM_EPOCHS):
  loss_total_gpu = tf.zeros([1,1], dtype=tf.float32)
  lt = 0
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(25, seed=epoch*(1234)).batch(20)
  for inputs, outputs in train_ds:
    preds = mlp_on_gpu.forward(inputs)
    loss_total_gpu = loss_total_gpu + mlp_on_gpu.loss(preds, outputs)
    lt = lt + mlp_on_gpu.loss(preds, outputs)
    mlp_on_gpu.backward(inputs, outputs)
  print('Number of Epoch = {} - Average MSE:= {}'.format(epoch + 1, np.sum(loss_total_gpu) / X_train.shape[0]))
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

Number of Epoch = 1 - Average MSE:= 5968.365
Number of Epoch = 2 - Average MSE:= 37.31454296875
Number of Epoch = 3 - Average MSE:= 19.48196875
Number of Epoch = 4 - Average MSE:= 12.4020673828125
Number of Epoch = 5 - Average MSE:= 8.905400390625
Number of Epoch = 6 - Average MSE:= 6.7273447265625
Number of Epoch = 7 - Average MSE:= 5.28943359375
Number of Epoch = 8 - Average MSE:= 4.5078193359375
Number of Epoch = 9 - Average MSE:= 3.793795654296875
Number of Epoch = 10 - Average MSE:= 3.2738134765625

Total time taken (in seconds): 4.08


## One Step Inference

In [ ]:
test_loss_total = tfe.Variable(0, dtype=tf.float32)
for inputs, outputs in test_ds:
  preds = mlp_on_default.forward(inputs)
  test_loss_total = test_loss_total + mlp_on_default.loss(preds, outputs)
print('Test MSE: {:.4f}'.format(test_loss_total.numpy() / X_train.shape[0]))

Test MSE: 11.5391
